In [1]:
# STEP 1: Install Required Libraries
!pip install pandas numpy sentence-transformers --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.7 MB/s eta 0:00:00


In [2]:
# STEP 2: Import Libraries
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("deeguy/wayfair-wands-dataset")

print("Path to dataset files:", path)

KeyboardInterrupt: 

In [4]:
products_df = pd.read_csv('/content/product.csv', sep='\t')
queries_df = pd.read_csv('/content/query.csv',sep='\t')
labels_df = pd.read_csv('/content/label.csv',sep='\t')


merged_df = labels_df.merge(queries_df, on='query_id', how='left')

merged_df = merged_df.merge(products_df, on='product_id', how='left')

# Preview the result
merged_df[['query', 'product_name', 'label']].head()


,query,product_name,label
0,salon chair,21.7 '' w waiting room chair with wood frame,Exact
1,salon chair,22.5 '' wide polyester side chair,Irrelevant
2,salon chair,24.4 '' w metal lounge chair with metal frame,Exact
3,salon chair,25 '' wide faux leather manual swivel standard...,Exact
4,salon chair,27.6 '' w antimicrobial leather seat waiting r...,Exact


In [5]:
print(products_df.columns)
print(queries_df.columns)
print(labels_df.columns)


Index(['product_id', 'product_name', 'product_class', 'category hierarchy',
       'product_description', 'product_features', 'rating_count',
       'average_rating', 'review_count'],
      dtype='object')
Index(['query_id', 'query', 'query_class'], dtype='object')
Index(['id', 'query_id', 'product_id', 'label'], dtype='object')


In [6]:
import re

def clean_text(text):
    text = str(text).lower()                              # lowercase
    text = re.sub(r'[^\w\s]', ' ', text)                  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()              # remove extra spaces
    return text

# Apply the cleaning function
merged_df['clean_query'] = merged_df['query'].apply(clean_text)
merged_df['clean_product'] = merged_df['product_name'].apply(clean_text)

# Preview cleaned data
merged_df[['query', 'clean_query', 'product_name', 'clean_product']].head()


,query,clean_query,product_name,clean_product
0,salon chair,salon chair,21.7 '' w waiting room chair with wood frame,21 7 w waiting room chair with wood frame
1,salon chair,salon chair,22.5 '' wide polyester side chair,22 5 wide polyester side chair
2,salon chair,salon chair,24.4 '' w metal lounge chair with metal frame,24 4 w metal lounge chair with metal frame
3,salon chair,salon chair,25 '' wide faux leather manual swivel standard...,25 wide faux leather manual swivel standard re...
4,salon chair,salon chair,27.6 '' w antimicrobial leather seat waiting r...,27 6 w antimicrobial leather seat waiting room...


In [7]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import torch

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Batch size for efficient encoding
batch_size = 256

# Batch encoding function
def batch_encode(texts):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch, convert_to_tensor=True)
        embeddings.extend(batch_embeddings)
    return embeddings

# Encode queries and products
query_embeddings = batch_encode(merged_df['clean_query'].tolist())
product_embeddings = batch_encode(merged_df['clean_product'].tolist())

# Compute cosine similarity row-wise
semantic_scores = [util.cos_sim(q, p).item() for q, p in zip(query_embeddings, product_embeddings)]
merged_df['semantic_score'] = semantic_scores


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 912/912 [01:12<00:00, 12.64it/s]


In [8]:
from sentence_transformers.util import cos_sim

# Calculate cosine similarity for each pair
semantic_scores = [cos_sim(q, p).item() for q, p in zip(query_embeddings, product_embeddings)]

# Add similarity scores to the DataFrame
merged_df['semantic_score'] = semantic_scores

# Preview results
merged_df[['query', 'product_name', 'label', 'semantic_score']].head()


,query,product_name,label,semantic_score
0,salon chair,21.7 '' w waiting room chair with wood frame,Exact,0.456427
1,salon chair,22.5 '' wide polyester side chair,Irrelevant,0.388587
2,salon chair,24.4 '' w metal lounge chair with metal frame,Exact,0.478494
3,salon chair,25 '' wide faux leather manual swivel standard...,Exact,0.370363
4,salon chair,27.6 '' w antimicrobial leather seat waiting r...,Exact,0.483570
